In [5]:
import re
import json
from pathlib import Path
from collections import Counter, defaultdict
from itertools import chain

import pandas as pd

from hebtokenizer import HebTokenizer

In [2]:
FOOTER = 'את הטקסט לעיל הפיקו מתנדבי פרויקט בן־יהודה באינטרנט.  הוא זמין תמיד בכתובת הבאה'

In [3]:
df = pd.read_csv('./public_domain_dump/pseudocatalogue.csv')

poetry_df = df.loc[df['genre'] == 'שירה', :]
poetry_df.loc[:,'content'] = (poetry_df['path']
                           .apply(lambda p: Path(f'./public_domain_dump/txt_stripped/{p}.txt').read_text())
                           .apply(lambda s: s.split(FOOTER)[0])
                           .str.strip())

poetry_df.loc[:, 'content'] = poetry_df['content'].str.replace('\t', '\n')

# perhaps_title_text_split = poetry_df['content'].str.split('\t', 1)
# perhaps_valid_split = (perhaps_title_text_split.apply(len) == 2)

# perhaps_title_text_split = perhaps_title_text_split[perhaps_valid_split]
# poetry_df = poetry_df[perhaps_valid_split]

# poetry_df['title'], poetry_df['text'] = perhaps_title_text_split.str[0], perhaps_title_text_split.str[1]

/Users/shlomi/Library/Caches/pypoetry/virtualenvs/markov-ben-yehuda-bS4ceNN1-py3.10/lib/python3.10/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/shlomi/Library/Caches/pypoetry/virtualenvs/markov-ben-yehuda-bS4ceNN1-py3.10/lib/python3.10/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [6]:
REPL_SPACE_BETWEEN_NEWLINE = '\n'
RE_SPACE_BETWEEN_NEWLINE = re.compile(r'\n[ ]+\n')
RE_LINES = re.compile('[\n/]')
RE_STANZA = re.compile('\n{3}')

def strip_filter(iterable):
    stripped = (item.strip() for item in iterable)
    filtered = (item for item in stripped if item)
    return filtered

def get_second(pairs):
    first, second = zip(*pairs)
    return second


def tokenizer(content):
    ht = HebTokenizer()
    
    content = content.replace('\t', '')
    content = RE_SPACE_BETWEEN_NEWLINE.sub(REPL_SPACE_BETWEEN_NEWLINE, content)
    stanzas = strip_filter(stanza.strip() for stanza in RE_STANZA.split(content))
    stanza_line = (strip_filter(RE_LINES.split(stanza)) for stanza in stanzas)

    tokens = list_join((list_join((get_second(ht.tokenize(line)) for line in stanza), 'NEWLINE')
              for stanza in stanza_line),'NEWSTANZA')
    
    return tokens


def list_join(iterable, sep):
    ret = []
    for item in iterable:
        ret.extend(item)
        ret.append(sep)
    return ret[:-1]


poetry_df.loc[:, 'tokens'] = poetry_df['content'].apply(tokenizer)

/Users/shlomi/Library/Caches/pypoetry/virtualenvs/markov-ben-yehuda-bS4ceNN1-py3.10/lib/python3.10/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [7]:
unigrams = Counter(chain(*poetry_df['tokens']))

bigrams = defaultdict(Counter)

for tokens in poetry_df['tokens']:
    for context, word in zip(tokens, tokens[1:]):
        bigrams[context][word] += 1

In [8]:
with open('unigrams.json', 'w') as f:
    json.dump(unigrams, f)
    
with open('bigrams.json', 'w') as f:
    json.dump(bigrams, f)